In [1]:
import tensorflow as tf
print(tf.__version__)


2.2.0


# Logistic Regression

### What is Logistic Regression?
1. Binary(Mulit-class) Classification
    - variable is either 0 or 1 (0 : positive, 1 : negative) <br>
      -> To start with machine learning, I must encode variable [0, 1]
      
2. Logistic vs Linear<br>
    - Logistic : target is Discrete(Counted), One Hot
    - Linear : target is Continous(Measured), Numeric
    
---

### How to solve?
1. Hypothesis Representation<br>
By start with binary classification, Y is only 0 or 1.
So we need new function<br>
=> Sigmoid function<br><br>

2. Sigmoid/Logistic Function<br>
X -> Linear Function -> Logistic Fucntion(Sigmoid) -> Decision Boundary -> Y is 0 or 1<br><br>

3. Decision Boundary<br>
If prediction >= 0.5, prediction == 1<br>
If prediction < 0.5, prediction == 0<br><br>

4. Cost Function <br>

In [2]:
x_train = [[1., 2.],
          [2., 3.],
          [3., 1.],
          [4., 3.],
          [5., 3.],
          [6., 2.]]

y_train = [[0.],
          [0.],
          [0.],
          [1.],
          [1.],
          [1.]]

x_test = [[5., 2.]]
y_test = [[1.]]

In [12]:
tf.compat.v1.enable_eager_execution()

# data 준비
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

# 가중치 준비
W = tf.Variable(tf.zeros([2, 1]), name = "weight")
b = tf.Variable(tf.zeros([1]), name = "bias")

# 로지스틱 회귀 식
def logistic_regression(features):
    hypothesis = tf.divide(1., 1. + tf.exp(tf.matmul(features, W) + b))
    return hypothesis

# 손실 함수 == 비용 함수
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels + tf.math.log(logistic_regression(features)) + (1 - labels)*tf.math.log(1-hypothesis))
            # tensorflow 1 에선 그냥 tf.log()
            # tensorflow 2 에선 tf.math.log()
    return cost

# 최적화
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# Gradient Descent 구현
def grad(features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features), features, labels)
    return tape.gradient(loss_value, [W, b])

# 학습
EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels in iter(dataset):   # 객체를 하나씩 하나씩 차례대로 불러오는 것
        grads = grad(features, labels)
        optimizer.apply_gradients(grads_and_vars = zip(grads, [W,b]))
        if step % 100 == 0:
            print("Iter : {}, Loss : {:.4f}".format(step, loss_fn(logistic_regression(features), features, labels)))

Iter : 0, Loss : 0.5197
Iter : 100, Loss : 0.3497
Iter : 200, Loss : 0.3387
Iter : 300, Loss : 0.3319
Iter : 400, Loss : 0.3273
Iter : 500, Loss : 0.3237
Iter : 600, Loss : 0.3208
Iter : 700, Loss : 0.3182
Iter : 800, Loss : 0.3159
Iter : 900, Loss : 0.3137
Iter : 1000, Loss : 0.3117


In [14]:
# 정확도 계산
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype = tf.float32))
    return accuracy

test_acc = accuracy_fn(logistic_regression(x_test), y_test)
print("TestSet Accuracy : {:.4f}".format(test_acc))

TestSet Accuracy : 1.0000
